In [2]:
import requests
from bs4 import BeautifulSoup
import re

In [6]:
def get_download_links(url, xpath_base):
  try:
    response = requests.get(url, timeout=10)
    response.raise_for_status() # Raise an exception for bad status codes (4xx or 5xx)
    soup = BeautifulSoup(response.content, "html.parser")

    download_links = []
    link_index = 1

    while link_index <= 28:
      matches = re.finditer(r"\[\d+\]", xpath_base)
      start, end = matches[4]
      xpath = xpath_base[:start] + f"[{link_index}]" + xpath_base[end:]

      link_element = soup.select_one(f'[data-testid="{xpath}"]')

      href = link_element.get("href")
      if href:
        download_links.append(href)

      link_index += 1

    return download_links

  except requests.exceptions.RequestException as e:
    print(f"Error fetching URL: {e}")
    return None
  except Exception as e:
    print(f"An error occured: {e}")
    return None


In [8]:
import requests
from bs4 import BeautifulSoup
import re
import time

def get_download_links(url, xpath_base):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    try:
        for attempt in range(3):  # Retry up to 3 times
            try:
                response = requests.get(url, timeout=10, headers=headers)
                response.raise_for_status()
                break
            except requests.exceptions.RequestException as e:
                print(f"Attempt {attempt + 1}: Failed to connect. Retrying...")
                time.sleep(5)
        else:
            print("Failed to connect after 3 attempts.")
            return None

        soup = BeautifulSoup(response.content, "html.parser")
        download_links = []
        link_index = 1

        while link_index <= 28:
            matches = list(re.finditer(r"\[\d+\]", xpath_base))
            if len(matches) > 4:  # Ensure the 5th match exists
                start, end = matches[4].span()
                xpath = xpath_base[:start] + f"[{link_index}]" + xpath_base[end:]
                print(f"Generated XPath: {xpath}")

                link_element = soup.select_one(f'[data-testid="{xpath}"]')

                if link_element:
                    href = link_element.get("href")
                    if href:
                        download_links.append(href)
            else:
                print("Invalid XPath pattern: Less than 5 matches found.")
                return None

            link_index += 1

        return download_links

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL: {e}")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Example usage
url = "https://ocw.sharif.ir/course/id/371"
xpath_base = "/html/body/div[1]/div/div/section/main/div/div[2]/div[3]/div[1]/div[2]/div/a[2]"
links = get_download_links(url, xpath_base)
print(links)


Attempt 1: Failed to connect. Retrying...
Attempt 2: Failed to connect. Retrying...
Attempt 3: Failed to connect. Retrying...
Failed to connect after 3 attempts.
None


In [7]:
url = "https://ocw.sharif.ir/course/id/371"
xpath_base = "/html/body/div[1]/div/div/section/main/div/div[2]/div[3]/div[1]/div[1]/div/a[2]"
result = get_download_links(url, xpath_base)

Error fetching URL: HTTPSConnectionPool(host='ocw.sharif.ir', port=443): Max retries exceeded with url: /course/id/371 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x797a8a664430>, 'Connection to ocw.sharif.ir timed out. (connect timeout=10)'))


In [5]:
print(result)

None
